### 3차원 좌표 변환 정리

 - 먼저 euler angle coordinate를 transformation coordinate로 바꾸기
 - 그 다음은 python에서 transformation matrix를 다루기 위해 list type으로 저장하는 방법을 봄
 - 저장된 list type을 numpy를 사용하여 numpy.array로 바꾸기
 - 이렇게 데이터 타입이 바뀐 transformation matrix를 numpy에서 지원하는 matrix 연산을 해보기
 
 > 추후 pandas도 같이 사용하여 수기 작성 없이 변환 및 계산이 가능하게 함수 생성 필요!

#### euler coordinate에서 transformation matrix로 변환하기

 - 변환은 언제나 error 누적이라는 부분을 늘 가지고 있음을 알아야 함
 - 먼저 euler coordinate에서 rotation term 부분만 연산을 진행
 - 그 이후, translation term에 해당하는 부분을 추가하는 방식을 사용
 - 아래에 작성한 함수는 data라는 변수를 받는 데 이는 euler angle의 6 term을 가지는 list 
 - 즉, 0~5의 list index에 각각 x, y, z, Rx, Ry, Rz가 입력됨
 - return되는 값은 transformation matrix로 numpy.array 형식
 
 > 참고자료 :
 > <https://en.wikipedia.org/wiki/Rotation_matrix>
 > <http://easyspin.org/easyspin/documentation/eulerangles.html>
 > <https://www.learnopencv.com/rotation-matrix-to-euler-angles/>

In [4]:
import math

def euler2mtx(data):
    posX = data[0]
    posY = data[1]
    posZ = data[2]
    pitch = data[3]
    roll = data[4]
    yaw = data[5]
    
    Rx = [[1, 0, 0, 0],
          [0, math.cos(pitch), math.sin(pitch), 0],
          [0, -1*math.sin(pitch), math.cos(pitch), 0],
          [0, 0, 0, 1]]
    
    Ry = [[math.cos(roll), 0, -1 * math.sin(roll), 0],
          [0, 1, 0, 0],
          [math.sin(roll), 0, math.cos(roll), 0],
          [0, 0, 0, 1]]
    
    Rz = [[math.cos(yaw), math.sin(yaw), 0, 0],
          [-1 * math.sin(yaw), math.cos(yaw), 0, 0],
          [0, 0, 1, 0],
          [0, 0, 0, 1]]
    
    mtxR = np.matmul(np.matmul(np.array(Rx), np.array(Ry)), np.array(Rz))
    
    mtxR[0][3] = posX
    mtxR[1][3] = posY
    mtxR[2][3] = posZ
    
    return mtxR

#### 3차원 transformation 연산

 - 먼저, csv의 transformation data를 받았다고 가정 --> 여기서는 이를 수기로 list type에 저장
 - transformation matrix가 저장되어 있는 list type을 python에서 matrix 연산을 쉽게하기 위해 numpy.array로 바꿈
 - numpy.array로 변경된 데이터를 np.matmul을 사용하여 matrix multiplication을 진행
 - 참고자료
 > 1. <https://docs.scipy.org/doc/numpy/reference/generated/numpy.matmul.html>
 > 2. <https://www.python-course.eu/matrix_arithmetic.php>

In [5]:
robot2base = [[-0.230246455124679,-0.343713471775025,0.91041069108691,377.636827588619],
              [0.044191197884157,-0.938274557846383,-0.343056956963818,413.839184030169],
              [0.972128487045339,-0.0387555069188734,0.231223484185981,294.024615805593],
              [0,0,0,1]]

ots2base_v1 = [-316.825444, 482.4823181, -1985.835834, -0.230162646, -0.056251684, 3.131292963]
ots2base_v2 = [32.75094299, 461.7471207, -2011.554492, -0.222212908, 0.107236162, 3.126904833]

ots2marker_v1 = [41.96472969, -208.2040459, -1617.433521, 0, -0.000147887, -1.514371941]
ots2marker_v2 = [322.9706802, -225.3924011, -1583.386017, 0.041197983, -0.021568373, -1.520555811]

marker2bone_v1 = [[-0.0331968143582344,-0.943108856678009,-0.330822706222534,-116.856120969405],
                  [0.167795315384865,0.321047723293304,-0.932079970836639,-130.746276136137],
                  [0.985262751579285,-0.0864525884389877,0.147591486573219,-88.9828105937332],
                  [0,0,0,1]]
marker2bone_v2 = [[-0.0542800202965736,-0.949741065502167,-0.308294713497162,-116.682082416504],
                  [0.184413388371468,0.293903678655624,-0.937876522541046,-130.989310617106],
                  [0.981348752975464,-0.1077616289258,0.159191861748695,-88.99862098117],
                  [0,0,0,1]]

bone2imp = [[-1.000000, 0.000000, 0.000000, -0.000046],
            [-0.000000, 0.992547, -0.121869, 8.286835],
            [0.000000, -0.121869, -0.992544, 3.332855],
            [0.000000, 0.000000, 0.000000, 1.000000]]

In [6]:
import numpy as np

robot2base_mtx = np.array(robot2base)

ots2base_v1_mtx = euler2mtx(ots2base_v1)
ots2base_v2_mtx = euler2mtx(ots2base_v2)

ots2marker_v1_mtx = euler2mtx(ots2marker_v1)
ots2marker_v2_mtx = euler2mtx(ots2marker_v2)

marker2bone_v1_mtx = np.array(marker2bone_v1)
marker2bone_v2_mtx = np.array(marker2bone_v2)

bone2imp_mtx = np.array(bone2imp)

In [7]:
def mulmulmtx(robot2base_mtx, ots2base_mtx, ots2marker_mtx, marker2bone_mtx, bone2imp_mtx):
    temp0 = np.matmul(robot2base_mtx, ots2base_mtx)
    temp1 = np.matmul(temp0, ots2marker_mtx)
    temp2 = np.matmul(temp1, marker2bone_mtx)
    temp3 = np.matmul(temp2, bone2imp_mtx)
    
    result = temp3
    
    return result

In [8]:
usrcrd_v1 = mulmulmtx(robot2base_mtx, np.linalg.inv(ots2base_v1_mtx), 
                      ots2marker_v1_mtx, marker2bone_v1_mtx, bone2imp_mtx)
usrcrd_v2 = mulmulmtx(robot2base_mtx, np.linalg.inv(ots2base_v2_mtx), 
                      ots2marker_v2_mtx, marker2bone_v2_mtx, bone2imp_mtx)

In [9]:
usrcrd_v1

array([[-8.87433374e-01, -3.47739459e-01, -3.02554109e-01,
         6.68117375e+02],
       [ 1.25971021e-01, -8.14374159e-01,  5.66503417e-01,
        -4.69797723e+02],
       [-4.43388432e-01,  4.64622094e-01,  7.66505447e-01,
        -5.82938699e+01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

In [10]:
usrcrd_v2

array([[-9.47792658e-01, -2.54787919e-01, -1.91760535e-01,
         6.55690760e+02],
       [ 1.31136618e-01, -8.59561044e-01,  4.93921611e-01,
        -4.55407620e+02],
       [-2.90675523e-01,  4.42989442e-01,  8.48094998e-01,
        -6.91018456e+01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])